## Import datasetu

In [ ]:
import deepchem as dc
import rdkit
import polars as pl
import numpy as np


2025-12-01 19:40:44.912905: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-01 19:40:45.223661: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-01 19:40:47.496175: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [2]:
dc.__version__

'2.6.0.dev'

## Przegląd danych

In [8]:
tasks, datasets, transformers = dc.molnet.load_chembl(
    featurizer=dc.feat.MACCSKeysFingerprint(),
    set='sparse',
    reload=False
)

Łączenie danych w jeden dataset

In [12]:
train_dataset, valid_dataset, test_dataset = datasets

full_dataset = dc.data.DiskDataset.merge(datasets)

In [14]:
# Pobranie nazw zadań (tasks)
tasks = full_dataset.get_task_names()

# Ekstrakcja danych NumPy z obiektu DeepChem Dataset
X_data = full_dataset.X
Y_data = full_dataset.y
W_data = full_dataset.w
mol_ids = full_dataset.ids

In [15]:
# Użycie maski wag (W) do stworzenia maski logicznej
# Tam, gdzie W=0.0, wartość Y jest nieznana.
# Użyjemy tej maski, aby jawnie ustawić NaN.
unknown_mask = (W_data == 0.0)

# Kopia danych Y, gdzie nieznane wartości są ustawione na NaN
Y_data_masked = Y_data.copy()
Y_data_masked[unknown_mask] = np.nan

# 1. Tworzenie Polars DataFrame z Y_data_masked
# Nazwy kolumn bierzemy z listy tasks
df_chembl_y = pl.DataFrame(
    {task: Y_data_masked[:, i] for i, task in enumerate(tasks)}
)

# 2. Dodanie identyfikatorów
df_chembl_y = df_chembl_y.with_columns(
    pl.Series("ID", mol_ids)
)

print("\n--- Fragment Data Frame Polars (Y) ---\n")
print(df_chembl_y.head())

NameError: name 'np' is not defined